## Outline:
### 1. Input transcripts and grammars

### 2. Sentencizer transcripts

### 3. Mapping sentences to each category

### 4. Calculate politeness score for each part

In [1]:
# Load packages
import nltk
import spacy

# Load customized packages
from customized_pos import *
from pos_helper import *
from aspect_matching import *
from politeness_helper import *

# Load spacy nlp model
nlp = spacy.load('en_core_web_lg')
# nlp.add_pipe(pos_postprocessor_pipe, name="pos_postprocessor", after='tagger')
nlp.add_pipe("pos_postprocessor_pipe", after='tagger')

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<function customized_pos.pos_postprocessor_pipe(doc)>

In [2]:
# Load standardizer
with open('other_collection/standardizer.pkl', 'rb') as f:
    standardizer = pickle.load(f)    
# Load custom model/data
xgb_model = pickle.load(open('model_collection/final_xgb_v2.sav', 'rb'))

### Section 1: Input transcripts and grammars

In [3]:
transcripts = """ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and im actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty"""

In [4]:
grammars = r""" 
    NP: {<PRP.*|DT|JJ|NN.*>+}
    PP: {<IN|TO|RP><NP|VB.*>} 
    VP: {<VB.*|RB.*>+<PP|NP>*}
    Sentence: {<UH>*<NP>+<MD>*<VP|PP|NP>+}
    Sentence: {<PP|VP|NP>+}
    Question: {<MD|WDT|DP|WRB|><MD>*<Sentence|NP|PP|VP>}
"""

### Section 2: Sentencizer transcripts

In [5]:
full_result = nlp_sentencizer(transcripts, grammars, nlp)
text_result = list(map(lambda x: x[1], full_result))
print("full result: ", full_result)
print("\n")
print("only text result: ", text_result)

(S
  (Sentence
    ya/UH
    hello/UH
    (NP good/JJ afternoon/NN)
    (VP speak/VBP (PP to/IN (NP nanny/NNP seah/NNP))))
  (Sentence
    please/UH
    (NP afternoon/NN)
    (VP miss/VB (NP nanny/NN my/PRP$ name/NN))
    (VP is/VBZ (NP jaguar/NNP shao/NNP)))
  and/CC
  (Sentence
    (NP i/PRP)
    (VP
      m/VBP
      actually/RB
      calling/VBG
      (PP from/IN (NP insurance/NN company/NN)))
    (VP
      miss/VBP
      (NP nanny/NN good/JJ time/NN)
      (PP to/TO speak/VB)
      (PP for/IN (NP a/DT while/NN this/DT)))
    (VP
      is/VBZ
      regarding/VBG
      (NP our/PRP$ partnership/NN charter_plus/NN members/NNS)))
  and/CC
  (Sentence
    (PP as/IN (NP a/DT result/NN))
    (PP of/IN (NP that/DT insurance/NN company/NN))
    (VP actually/RB formulated/VBD (NP a/DT))
    (VP
      very/RB
      (NP
        special/JJ
        thirtieth/JJ
        anniversay/NNP
        insurance/NNP
        bundle/NNP))
    (VP right/RB called/VBN (NP the/DT i/NNP)))
  thirty/CD)
full resu

### Section 3: Mapping sentences to each category (standard greetings, state purpose, ask for permission to speak)

In [6]:
category_ls = batch_match_category(text_result, nlp, "greeting")

[0.78770024, 0.54077864, 0.78323364]
[0.6406816, 0.66061425, 0.6163679]
[0.87054694, 0.90663826, 0.7614186]
[0.7013029, 0.9084571, 0.588287]


In [7]:
category_ls

[('ask_for_permission', 'ya hello good afternoon speak to nanny seah'),
 ('purpose_of_call', 'please afternoon miss nanny my name is jaguar shao and'),
 ('purpose_of_call',
  'i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and'),
 ('purpose_of_call',
  'as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty')]

In [9]:
new_category_ls = nlp_aspect_matching(text_result, nlp, 'greeting', True)
new_category_ls

[('ask_for_permission', 'ya hello good afternoon speak to nanny seah'),
 ('purpose_of_call', 'please afternoon miss nanny my name is jaguar shao and'),
 ('purpose_of_call',
  ' i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty')]

### Section 4: Calculate politeness score for each part

In [10]:
politeness_ls = nlp_politeness(new_category_ls, xgb_model, nlp, standardizer, "int", 0.4)
politeness_ls

[('ask_for_permission',
  'ya hello good afternoon speak to nanny seah',
  'polite'),
 ('purpose_of_call',
  'please afternoon miss nanny my name is jaguar shao and',
  'polite'),
 ('purpose_of_call',
  ' i m actually calling from insurance company miss nanny good time to speak for a while this is regarding our partnership charter_plus members and as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty',
  'polite')]

### Print result

In [14]:
for category, sentence, politeness in politeness_ls:
    print("Current Category: ", category)
    print("Current Sentence: ", sentence)
    print("Current Politeness Score: ", politeness)
    print("\n")

Current Category:  opening
Current Sentence:  ya hello good afternoon speak to nanny seah please afternoon miss nanny my name is jaguar shao and
Current Politeness Score:  polite


Current Category:  purpose_of_call
Current Sentence:   as a result of that insurance company actually formulated a very special thirtieth anniversay insurance bundle right called the i thirty
Current Politeness Score:  polite


